Transformation is done based on this document -- [Link](https://docs.google.com/document/d/1_wt-wO18sa5iQBy3b-WpSHDjCqRl3MeN67N1v3dTZtQ/edit#heading=h.usgit3agfa62)

In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

In [16]:
bharat_maps_villages = gpd.read_file('/home/krishna/IDS-DRR-Data-Pipeline/Maps/bharatmapnew_assam_villages.geojson')

In [17]:
mission_antyodaya_df = pd.read_excel('/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/MissionAntyodaya2020_Assam.xlsx')

In [18]:
mission_antyodaya_df.shape

(26673, 198)

In [19]:
mission_antyodaya_geometry = [
        Point(xy) for xy in zip(
            mission_antyodaya_df.village_longitude,
            mission_antyodaya_df.village_latitude
        )
    ]

In [20]:
gdf_points = GeoDataFrame(mission_antyodaya_df, geometry=mission_antyodaya_geometry)

In [21]:
gdf_polygons = GeoDataFrame.from_file('/home/krishna/IDS-DRR-Data-Pipeline/Maps/Assam_Revenue_Circles/assam_revenue_circle_nov2022.geojson')

In [22]:
gdf_points.crs = gdf_polygons.crs

In [30]:
result = gpd.sjoin(gdf_points, gdf_polygons, how="left", op="within")

/home/krishna/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [34]:
result.tail()

,state_code,state_name,district_code,district_name,sub_district_code,sub_district_name,block_code,block_name,gp_code,gp_name,...,geometry,index_right,revenue_ci,district_3,object_id,district_1,revenue_cr,HQ,area,are_new
26668,18,ASSAM,282,CACHAR,2100,Lakhipur,2514,LAKHIPUR,105851,FULERTAL,...,POINT (93.00465 24.79279),87.0,Lakhipur,CACHAR,188.0,Cachar,Lakhipur,None,754436388.7,754.0
26669,18,ASSAM,282,CACHAR,2100,Lakhipur,2514,LAKHIPUR,105851,FULERTAL,...,POINT (93.02450 24.79279),87.0,Lakhipur,CACHAR,188.0,Cachar,Lakhipur,None,754436388.7,754.0
26670,18,ASSAM,282,CACHAR,2100,Lakhipur,2514,LAKHIPUR,105851,FULERTAL,...,POINT (93.01817 24.81081),87.0,Lakhipur,CACHAR,188.0,Cachar,Lakhipur,None,754436388.7,754.0
26671,18,ASSAM,282,CACHAR,2100,Lakhipur,2514,LAKHIPUR,105851,FULERTAL,...,POINT (93.01817 24.81081),87.0,Lakhipur,CACHAR,188.0,Cachar,Lakhipur,None,754436388.7,754.0
26672,18,ASSAM,282,CACHAR,2096,Katigora,2511,BORKHOLA,105815,CHANDRANATHPUR,...,POINT (92.73985 24.93774),84.0,Silchar,CACHAR,185.0,Cachar,Silchar,y,837931796.0,838.0


In [24]:
untagged = result[result.revenue_ci.isnull()]
tagged = result.dropna(subset=['revenue_ci'])

In [25]:
untagged_bm = bharat_maps_villages[bharat_maps_villages.vil_lgd.isin(untagged.village_code.to_list())]
untagged_bm['geometry'] = untagged_bm.geometry.centroid
untagged_bm = untagged_bm.to_crs('EPSG:4326')
untagged_bm = gpd.sjoin(untagged_bm, gdf_polygons, how="left", op="within")

/home/krishna/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/krishna/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [26]:
untagged = pd.merge(untagged.drop(['object_id','revenue_ci'],axis=1),untagged_bm[['vil_lgd','object_id','revenue_ci']],
        left_on='village_code', right_on='vil_lgd')[untagged.columns]

In [27]:
tagged.shape

(25891, 208)

In [28]:
untagged.shape

(514, 208)

In [29]:
result = pd.concat([tagged,untagged])

In [16]:
result.to_csv('/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/MissionAntyodaya2020_Assam_taggedRC.csv')

In [17]:
result[['gp_code','village_code','object_id','revenue_ci','net_sown_area_in_hac','total_hhd',
        'availablility_hours_of_domestic_electricity', 'availability_of_telephone_services',
       'total_hhd_having_piped_water_connection', 'total_hhd_not_having_sanitary_latrines']].to_csv(
    '/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/MissionAntyodaya2020_Assam_vul.csv', index=False)

# Number of households in each urban area

In [5]:
assam_urban_shapes = gpd.read_file('/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/assam_urban_shapes/assam_urban_pd_grt_2000.shp')
assam_urban_shapes

,area,id,revenue_ci,district_3,object_id,district_1,revenue_cr,longitude,latitude,geometry
0,2.320,1,Mankachar,SOUTH SALMARA MANCACHAR,257,South Salmara Mancachar,Mankachar,89.863913,25.528749,"POLYGON ((89.85708 25.52458, 89.85708 25.53292..."
1,85.747,2,Mankachar,SOUTH SALMARA MANCACHAR,257,South Salmara Mancachar,Mankachar,89.888590,25.698188,"POLYGON ((89.88208 25.64125, 89.88208 25.64958..."
2,0.773,3,Mankachar,SOUTH SALMARA MANCACHAR,257,South Salmara Mancachar,Mankachar,89.928627,25.646718,"POLYGON ((89.92375 25.64125, 89.92375 25.64958..."
3,6.931,4,Dhubri (Pt),DHUBRI,112,Dhubri,Dhubri (Pt),89.979711,26.029384,"POLYGON ((89.96542 26.03292, 89.97375 26.03292..."
4,4.618,5,Dhubri (Pt),DHUBRI,112,Dhubri,Dhubri (Pt),89.980141,26.092077,"POLYGON ((89.98208 26.09125, 89.97375 26.09125..."
...,...,...,...,...,...,...,...,...,...,...
176,5.318,177,Doom Dooma,TINSUKIA,155,Tinsukia,Doom Dooma,95.567652,27.598153,"POLYGON ((95.55708 27.60792, 95.56542 27.60792..."
177,0.760,178,Doom Dooma,TINSUKIA,155,Tinsukia,Doom Dooma,95.659610,27.628244,"POLYGON ((95.66542 27.62458, 95.65708 27.62458..."
178,0.759,179,Doom Dooma,TINSUKIA,155,Tinsukia,Doom Dooma,95.558260,27.674442,"POLYGON ((95.55708 27.66625, 95.55708 27.67458..."
179,0.759,180,Doom Dooma,TINSUKIA,155,Tinsukia,Doom Dooma,95.592361,27.754740,"POLYGON ((95.59042 27.74958, 95.59042 27.75792..."


In [6]:
# TOTAL POPULATION IN EACH URBAN AREA
import rasterio
import rasterstats

worldpop_raster = rasterio.open('/home/krishna/IDS-DRR-Data-Pipeline/Sources/WORLDPOP/data/population_counts/assam_ppp_2020_UNadj.tif')
worldpop_raster_array = worldpop_raster.read(1)

sum_dicts = rasterstats.zonal_stats(assam_urban_shapes.to_crs(worldpop_raster.crs),
                                     worldpop_raster_array,
                                     affine= worldpop_raster.transform,
                                     stats= ['sum'],
                                     nodata=worldpop_raster.nodata,
                                     geojson_out = True)

dfs = []
for rc in sum_dicts:
    dfs.append(pd.DataFrame([rc['properties']]))

pop_zonal_stats_df = pd.concat(dfs).reset_index(drop=True)
pop_zonal_stats_df = pop_zonal_stats_df.rename(columns={'sum':'sum_population'})

In [7]:
# Divide population by 4.3 for households
pop_zonal_stats_df['urban_hhd'] = pop_zonal_stats_df['sum_population']/4.3

In [8]:
urban_hhd = pop_zonal_stats_df.groupby('object_id')[['urban_hhd']].sum().reset_index()
urban_hhd

,object_id,urban_hhd
0,101,1741.001885
1,103,2355.745481
2,104,11808.795365
3,108,7669.858285
4,111,370.653786
...,...,...
85,271,3334.718841
86,272,1212.109318
87,273,474.167197
88,276,32270.481638


In [22]:
urban_hhd['urban_electricity'] = urban_hhd['urban_hhd']*20

In [23]:
urban_hhd['urban_tele'] = urban_hhd['urban_hhd']*3

In [24]:
urban_hhd['urban_hhd_pipe'] = urban_hhd['urban_hhd']*0.1873

In [25]:
urban_hhd['urban_no_sanitation'] = urban_hhd['urban_hhd']*0.063

# Vul variables

In [2]:
vul_df = pd.read_csv('/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/MissionAntyodaya2020_Assam_vul.csv')

In [3]:
vul_df = vul_df.rename(columns={'total_hhd':'rural_hhd'})

## net_sown_area_in_hac

In [28]:
net_sown_area_in_hac_rc = vul_df.groupby(['object_id'])[['net_sown_area_in_hac']].sum()

In [29]:
net_sown_area_in_hac_rc

,net_sown_area_in_hac
object_id,
101.0,32583.05000
102.0,2199.40000
103.0,18388.48500
104.0,21680.64300
105.0,554.41709
...,...
277.0,7845.35000
278.0,6195.96000
279.0,4235.76000


## Rural - Urban Households in each rc

In [12]:
rural_hhd = vul_df.groupby('object_id')[['rural_hhd']].sum().reset_index()
rural_hhd

,object_id,rural_hhd
0,101.0,60683
1,102.0,5047
2,103.0,37355
3,104.0,57306
4,105.0,10601
...,...,...
170,277.0,53223
171,278.0,38275
172,279.0,38376
173,280.0,17201


In [15]:
vul_df[vul_df.object_id == 258]

,gp_code,village_code,object_id,revenue_ci,net_sown_area_in_hac,rural_hhd,availablility_hours_of_domestic_electricity,availability_of_telephone_services,total_hhd_having_piped_water_connection,total_hhd_not_having_sanitary_latrines


In [9]:
total_hhd = urban_hhd.merge(rural_hhd, how='outer', on='object_id')
total_hhd

,object_id,urban_hhd,rural_hhd
0,101.0,1741.001885,60683.0
1,103.0,2355.745481,37355.0
2,104.0,11808.795365,57306.0
3,108.0,7669.858285,1133.0
4,111.0,370.653786,56285.0
...,...,...,...
171,275.0,NaN,38242.0
172,277.0,NaN,53223.0
173,278.0,NaN,38275.0
174,280.0,NaN,17201.0


In [10]:
total_hhd = total_hhd.fillna(0)
total_hhd['total_hhd']=total_hhd['rural_hhd']+total_hhd['urban_hhd']

In [11]:
total_hhd

,object_id,urban_hhd,rural_hhd,total_hhd
0,101.0,1741.001885,60683.0,62424.001885
1,103.0,2355.745481,37355.0,39710.745481
2,104.0,11808.795365,57306.0,69114.795365
3,108.0,7669.858285,1133.0,8802.858285
4,111.0,370.653786,56285.0,56655.653786
...,...,...,...,...
171,275.0,0.000000,38242.0,38242.000000
172,277.0,0.000000,53223.0,53223.000000
173,278.0,0.000000,38275.0,38275.000000
174,280.0,0.000000,17201.0,17201.000000


## availablility_hours_of_domestic_electricity

In [34]:
# Map category to mean electricity available
electricity_dict = {1:2.5, 2:6, 3:10, 4:12, 5:0}

In [35]:
elect_df = vul_df.replace({"availablility_hours_of_domestic_electricity": electricity_dict})[['object_id','rural_hhd','availablility_hours_of_domestic_electricity']]

In [36]:
elect_df['rural_electricity'] = elect_df['rural_hhd']*elect_df['availablility_hours_of_domestic_electricity']

In [37]:
elect_df = elect_df.groupby('object_id')[['rural_hhd','rural_electricity']].sum().reset_index()

In [38]:
elect_df

,object_id,rural_hhd,rural_electricity
0,101.0,60683,345703.0
1,102.0,5047,24946.5
2,103.0,37355,316646.5
3,104.0,57306,408932.0
4,105.0,10601,95102.5
...,...,...,...
170,277.0,53223,542716.0
171,278.0,38275,316328.0
172,279.0,38376,329878.0
173,280.0,17201,159390.0


In [39]:
rc_electricity = elect_df.merge(urban_hhd, on='object_id', how='outer')

In [40]:
rc_electricity = rc_electricity.fillna(0)

In [41]:
rc_electricity['total_electricity'] = rc_electricity['rural_electricity'] + rc_electricity['urban_electricity']

In [42]:
rc_electricity['total_hhd'] = rc_electricity['rural_hhd'] + rc_electricity['urban_hhd']

In [43]:
rc_electricity

,object_id,rural_hhd,rural_electricity,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,total_electricity,total_hhd
0,101.0,60683.0,345703.0,1741.001885,34820.037700,5223.005655,326.089653,109.683119,380523.037700,62424.001885
1,102.0,5047.0,24946.5,0.000000,0.000000,0.000000,0.000000,0.000000,24946.500000,5047.000000
2,103.0,37355.0,316646.5,2355.745481,47114.909611,7067.236442,441.231129,148.411965,363761.409611,39710.745481
3,104.0,57306.0,408932.0,11808.795365,236175.907295,35426.386094,2211.787372,743.954108,645107.907295,69114.795365
4,105.0,10601.0,95102.5,0.000000,0.000000,0.000000,0.000000,0.000000,95102.500000,10601.000000
...,...,...,...,...,...,...,...,...,...,...
171,278.0,38275.0,316328.0,0.000000,0.000000,0.000000,0.000000,0.000000,316328.000000,38275.000000
172,279.0,38376.0,329878.0,2016.814022,40336.280432,6050.442065,377.749266,127.059283,370214.280432,40392.814022
173,280.0,17201.0,159390.0,0.000000,0.000000,0.000000,0.000000,0.000000,159390.000000,17201.000000
174,281.0,22038.0,207932.5,0.000000,0.000000,0.000000,0.000000,0.000000,207932.500000,22038.000000


In [44]:
rc_electricity['avg_electricity'] = rc_electricity['total_electricity']/rc_electricity['total_hhd']

In [45]:
rc_electricity

,object_id,rural_hhd,rural_electricity,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,total_electricity,total_hhd,avg_electricity
0,101.0,60683.0,345703.0,1741.001885,34820.037700,5223.005655,326.089653,109.683119,380523.037700,62424.001885,6.095781
1,102.0,5047.0,24946.5,0.000000,0.000000,0.000000,0.000000,0.000000,24946.500000,5047.000000,4.942837
2,103.0,37355.0,316646.5,2355.745481,47114.909611,7067.236442,441.231129,148.411965,363761.409611,39710.745481,9.160277
3,104.0,57306.0,408932.0,11808.795365,236175.907295,35426.386094,2211.787372,743.954108,645107.907295,69114.795365,9.333861
4,105.0,10601.0,95102.5,0.000000,0.000000,0.000000,0.000000,0.000000,95102.500000,10601.000000,8.971088
...,...,...,...,...,...,...,...,...,...,...,...
171,278.0,38275.0,316328.0,0.000000,0.000000,0.000000,0.000000,0.000000,316328.000000,38275.000000,8.264611
172,279.0,38376.0,329878.0,2016.814022,40336.280432,6050.442065,377.749266,127.059283,370214.280432,40392.814022,9.165350
173,280.0,17201.0,159390.0,0.000000,0.000000,0.000000,0.000000,0.000000,159390.000000,17201.000000,9.266322
174,281.0,22038.0,207932.5,0.000000,0.000000,0.000000,0.000000,0.000000,207932.500000,22038.000000,9.435180


## availability_of_telephone_services

In [46]:
telephone_df = vul_df.copy()

In [47]:
telephone_df['rural_tele'] = vul_df['rural_hhd']*vul_df['availability_of_telephone_services']

In [48]:
telephone_df = telephone_df[['object_id', 'rural_hhd', 'rural_tele']]

In [49]:
telephone_df = telephone_df.groupby('object_id')[['rural_hhd','rural_tele']].sum().reset_index()

In [50]:
rc_tele = telephone_df.merge(urban_hhd[['object_id', 'urban_hhd','urban_tele']], on='object_id', how='outer')

In [51]:
rc_tele = rc_tele.fillna(0)

In [52]:
rc_tele['total_tele'] = rc_tele['rural_tele'] + rc_tele['urban_tele']

In [53]:
rc_tele['total_hhd'] = rc_tele['rural_hhd'] + rc_tele['urban_hhd']

In [54]:
rc_tele['avg_tele'] = round(rc_tele['total_tele']/rc_tele['total_hhd'])

In [55]:
rc_tele

,object_id,rural_hhd,rural_tele,urban_hhd,urban_tele,total_tele,total_hhd,avg_tele
0,101.0,60683.0,134603.0,1741.001885,5223.005655,139826.005655,62424.001885,2.0
1,102.0,5047.0,10803.0,0.000000,0.000000,10803.000000,5047.000000,2.0
2,103.0,37355.0,82625.0,2355.745481,7067.236442,89692.236442,39710.745481,2.0
3,104.0,57306.0,145858.0,11808.795365,35426.386094,181284.386094,69114.795365,3.0
4,105.0,10601.0,22938.0,0.000000,0.000000,22938.000000,10601.000000,2.0
...,...,...,...,...,...,...,...,...
171,278.0,38275.0,88983.0,0.000000,0.000000,88983.000000,38275.000000,2.0
172,279.0,38376.0,81939.0,2016.814022,6050.442065,87989.442065,40392.814022,2.0
173,280.0,17201.0,47632.0,0.000000,0.000000,47632.000000,17201.000000,3.0
174,281.0,22038.0,48027.0,0.000000,0.000000,48027.000000,22038.000000,2.0


## total_hhd_having_piped_water_connection

In [56]:
pipe_df = vul_df.copy()

In [57]:
pipe_df = pipe_df.groupby('object_id')[['total_hhd_having_piped_water_connection']].sum().reset_index()

In [58]:
pipe_df = pipe_df.merge(urban_hhd[['object_id','urban_hhd_pipe']], on='object_id', how='outer')

In [59]:
pipe_df = pipe_df.fillna(0)

In [60]:
pipe_df['rc_piped_hhds'] = pipe_df['urban_hhd_pipe'] + pipe_df['total_hhd_having_piped_water_connection']

In [61]:
pipe_df = pipe_df.merge(total_hhd[['object_id','total_hhd']], on='object_id')

In [62]:
pipe_df['rc_piped_hhds_pct'] = 100*pipe_df['rc_piped_hhds']/pipe_df['total_hhd']

In [63]:
pipe_df

,object_id,total_hhd_having_piped_water_connection,urban_hhd_pipe,rc_piped_hhds,total_hhd,rc_piped_hhds_pct
0,101.0,259.0,326.089653,585.089653,62424.001885,0.937283
1,102.0,2.0,0.000000,2.000000,5047.000000,0.039628
2,103.0,295.0,441.231129,736.231129,39710.745481,1.853985
3,104.0,108.0,2211.787372,2319.787372,69114.795365,3.356427
4,105.0,0.0,0.000000,0.000000,10601.000000,0.000000
...,...,...,...,...,...,...
171,278.0,1297.0,0.000000,1297.000000,38275.000000,3.388635
172,279.0,2435.0,377.749266,2812.749266,40392.814022,6.963489
173,280.0,360.0,0.000000,360.000000,17201.000000,2.092902
174,281.0,1722.0,0.000000,1722.000000,22038.000000,7.813776


## total_hhd_not_having_sanitary_latrines

In [64]:
nosanitation_df = vul_df.copy()

In [65]:
nosanitation_df = nosanitation_df.groupby('object_id')[['total_hhd_not_having_sanitary_latrines']].sum().reset_index()

In [66]:
nosanitation_df = nosanitation_df.merge(urban_hhd[['object_id','urban_no_sanitation']], on='object_id', how='outer')

In [67]:
nosanitation_df = nosanitation_df.fillna(0)

In [68]:
nosanitation_df['rc_nosanitation_hhds'] = nosanitation_df['urban_no_sanitation'] + nosanitation_df['total_hhd_not_having_sanitary_latrines']

In [69]:
nosanitation_df = nosanitation_df.merge(total_hhd[['object_id','total_hhd']], on='object_id')

In [70]:
nosanitation_df['rc_nosanitation_hhds_pct'] = 100*nosanitation_df['rc_nosanitation_hhds']/nosanitation_df['total_hhd']

In [71]:
nosanitation_df

,object_id,total_hhd_not_having_sanitary_latrines,urban_no_sanitation,rc_nosanitation_hhds,total_hhd,rc_nosanitation_hhds_pct
0,101.0,12478.0,109.683119,12587.683119,62424.001885,20.164813
1,102.0,734.0,0.000000,734.000000,5047.000000,14.543293
2,103.0,7835.0,148.411965,7983.411965,39710.745481,20.103909
3,104.0,14911.0,743.954108,15654.954108,69114.795365,22.650655
4,105.0,2962.0,0.000000,2962.000000,10601.000000,27.940760
...,...,...,...,...,...,...
171,278.0,12417.0,0.000000,12417.000000,38275.000000,32.441541
172,279.0,3324.0,127.059283,3451.059283,40392.814022,8.543746
173,280.0,4878.0,0.000000,4878.000000,17201.000000,28.358816
174,281.0,1367.0,0.000000,1367.000000,22038.000000,6.202922


# ANTYODAYA MASTER

In [72]:
antyodaya_master_df = net_sown_area_in_hac_rc.reset_index()

In [73]:
antyodaya_master_df = antyodaya_master_df.merge(rc_electricity[['object_id', 'avg_electricity']], how='outer')

In [74]:
antyodaya_master_df = antyodaya_master_df.merge(rc_tele[['object_id', 'avg_tele']])

In [75]:
antyodaya_master_df = antyodaya_master_df.merge(pipe_df[['object_id', 'rc_piped_hhds_pct']])

In [76]:
antyodaya_master_df = antyodaya_master_df.merge(nosanitation_df[['object_id','rc_nosanitation_hhds_pct']])


In [77]:
antyodaya_master_df

,object_id,net_sown_area_in_hac,avg_electricity,avg_tele,rc_piped_hhds_pct,rc_nosanitation_hhds_pct
0,101.0,32583.05000,6.095781,2.0,0.937283,20.164813
1,102.0,2199.40000,4.942837,2.0,0.039628,14.543293
2,103.0,18388.48500,9.160277,2.0,1.853985,20.103909
3,104.0,21680.64300,9.333861,3.0,3.356427,22.650655
4,105.0,554.41709,8.971088,2.0,0.000000,27.940760
...,...,...,...,...,...,...
171,278.0,6195.96000,8.264611,2.0,3.388635,32.441541
172,279.0,4235.76000,9.165350,2.0,6.963489,8.543746
173,280.0,5732.95000,9.266322,3.0,2.092902,28.358816
174,281.0,10909.71000,9.435180,2.0,7.813776,6.202922


In [78]:
antyodaya_master_df = pd.merge(gdf_polygons,antyodaya_master_df,on='object_id', how='outer')

In [79]:
antyodaya_master_df[antyodaya_master_df.avg_electricity.isnull()]

,revenue_ci,district_3,object_id,district_1,revenue_cr,HQ,area,are_new,geometry,net_sown_area_in_hac,avg_electricity,avg_tele,rc_piped_hhds_pct,rc_nosanitation_hhds_pct
48,Subansiri (Pt-II),DHEMAJI,149,Dhemaji,Subansiri (Pt-II),None,1.552108e+08,155,"MULTIPOLYGON (((94.25283 27.63639, 94.25352 27...",NaN,NaN,NaN,NaN,NaN
141,Pathorighat (Pt),UDALGURI,242,Udalguri,Pathorighat (Pt),None,2.314177e+06,2,"MULTIPOLYGON (((91.81536 26.52575, 91.81724 26...",NaN,NaN,NaN,NaN,NaN
142,Mangaldoi (Pt),UDALGURI,243,Udalguri,Mangaldoi (Pt),None,6.293735e+06,6,"MULTIPOLYGON (((92.04131 26.54834, 92.04467 26...",NaN,NaN,NaN,NaN,NaN
157,Bajali (Pt),TAMULPUR,258,Baksa,Bajali (Pt),None,1.784417e+07,18,"MULTIPOLYGON (((91.61612 26.51646, 91.61559 26...",NaN,NaN,NaN,NaN,NaN


In [80]:
antyodaya_master_df = antyodaya_master_df.merge(total_hhd[['object_id', 'total_hhd']], on='object_id')
antyodaya_master_df['total_hhd'] = antyodaya_master_df['total_hhd'].apply(lambda x: round(x,0))

In [81]:
antyodaya_master_df.drop(['geometry',
                           'district_1', 'revenue_cr', 'HQ', 'area', 
                           'are_new'],axis=1).to_csv('/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/variables/antyodaya/antyodaya_variables.csv', index=False)

In [82]:
antyodaya_master_df

,revenue_ci,district_3,object_id,district_1,revenue_cr,HQ,area,are_new,geometry,net_sown_area_in_hac,avg_electricity,avg_tele,rc_piped_hhds_pct,rc_nosanitation_hhds_pct,total_hhd
0,Gossaigaon (Pt),KOKRAJHAR,101,Kokrajhar,Gossaigaon (Pt),None,1.068572e+09,1069,"MULTIPOLYGON (((90.14118 26.74010, 90.15342 26...",32583.05000,6.095781,2.0,0.937283,20.164813,62424.0
1,Bhowraguri,KOKRAJHAR,102,Kokrajhar,Bhawraguri,None,1.588926e+08,159,"MULTIPOLYGON (((90.09811 26.46455, 90.10012 26...",2199.40000,4.942837,2.0,0.039628,14.543293,5047.0
2,Dotoma,KOKRAJHAR,103,Kokrajhar,Dotoma,None,3.039662e+08,304,"MULTIPOLYGON (((90.19819 26.63409, 90.21635 26...",18388.48500,9.160277,2.0,1.853985,20.103909,39711.0
3,Kokrajhar (Pt),KOKRAJHAR,104,Kokrajhar,Kokrajhar (Pt),y,9.902467e+08,990,"MULTIPOLYGON (((90.33590 26.86280, 90.33610 26...",21680.64300,9.333861,3.0,3.356427,22.650655,69115.0
4,Bagribari (Pt),KOKRAJHAR,105,Kokrajhar,Bagribari (Pt),None,2.810115e+08,281,"MULTIPOLYGON (((89.99553 26.35026, 89.99717 26...",554.41709,8.971088,2.0,0.000000,27.940760,10601.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,Sapekhati,CHARAIDEO,278,Charaideo,Sapekhati,None,3.943284e+08,394,"MULTIPOLYGON (((95.10936 27.12364, 95.10927 27...",6195.96000,8.264611,2.0,3.388635,32.441541,38275.0
172,Sonari,CHARAIDEO,279,Charaideo,Sonari,y,3.845428e+08,385,"MULTIPOLYGON (((94.95742 27.06354, 94.95822 27...",4235.76000,9.165350,2.0,6.963489,8.543746,40393.0
173,Ujani Majuli,MAJULI,280,Majuli,Ujani Majuli,None,3.221681e+08,322,"MULTIPOLYGON (((94.33804 26.89186, 94.33850 26...",5732.95000,9.266322,3.0,2.092902,28.358816,17201.0
174,Majuli,MAJULI,281,Majuli,Majuli,None,6.484667e+08,648,"MULTIPOLYGON (((94.56453 27.18172, 94.56755 27...",10909.71000,9.435180,2.0,7.813776,6.202922,22038.0
